#### This Code is designed to simultaneously sweep FG4 and FG6, in order to change the amplitude when switching between the (0,2) and (1,1) state. 

# General Imports

In [30]:
%config IPCompleter.greedy=True

# import matplotlib.pyplot as plt
import numpy as np
import time
import zhinst.core
import laboneq
from laboneq.simple import *
import ctypes
import helpers
import matplotlib.pyplot as plt
from picosdk.ps5000a import ps5000a as ps
from picosdk.functions import adc2mV, assert_pico_ok, mV2adc
import picoscope_module as pm
#install_token("LabOneQ.AllQuantumInOne.")
DO_EMULATION = False # run in emulation mode by default

print(laboneq.__version__)

2.11.0


# Sampling card setup

## Device Setup

Picoscope and HDAWG

In [31]:
timebase = 900 #628 for TwoLevelPulse
pico_sampling_rate = (timebase - 3) / 62500000
preTriggerSamples = 0 #10 for TwoLevelPulse 
postTriggerSamples = int(50e-3/pico_sampling_rate) #or int(np.ceil(20e-3/pico_sampling_rate))
print(postTriggerSamples)
TriggerLevel=1

maxSamples = preTriggerSamples + postTriggerSamples
ready = ctypes.c_int16(0)
check = ctypes.c_int16(0)
maxADC = ctypes.c_int16() #what's the difference between having no handle and a handle of 0 

chandle, status = pm.initialize_picoscope()
chARange, chBRange, status = pm.setup_channels(chandle)
# find maximum ADC count value

pm.setup_trigger(chandle, chARange, chBRange, maxADC, TriggerLevel)

# Get timebase information

3483


{'maximumValue': 0,
 'setTriggerChannelPropertiesV2': 0,
 'setTriggerChannelConditionsV2': 0,
 'setTriggerChannelDirections': 0}

Only run, if you want to disconnect the scope and plan on running the previous cell again 

In [32]:
# Stop the scope
#status["stop"] = ps.ps5000aStop(chandle)
#assert_pico_ok(status["stop"])

# Close unit Disconnect the scope
#status["close"]=ps.ps5000aCloseUnit(chandle)
#assert_pico_ok(status["close"])

## Create device setup -HDAWG

In [33]:
descriptor="""
instrument_list:
  HDAWG:
  - address: DEV8721
    uid: device_hdawg
    interface: usb
connections:
  device_hdawg:
    - rf_signal: q0/fg8_line
      ports: [SIGOUTS/0]
    - rf_signal: q0/fg9_line
      ports: [SIGOUTS/1]
"""
#Change back to to SIGOUTS/2 and SIGOUTS/3
#And change fg8 and fg9
device_setup = DeviceSetup.from_descriptor(
    descriptor,
    server_host="127.0.0.1",
    server_port="8004",
    setup_name="ZI_HDAWG",
)

## Define parameters for experiment

In [34]:
# Wister's Hack, make sure to change sampling rate when running PulseTimeResolved code 

sampling_index = 13 # sampling rate is 2.4GHz/2^13, which is the minumium sampling rate HDAWG can go
daq = zhinst.core.ziDAQServer('127.0.0.1', 8004, 6) # Connect to awg with zhinst driver
daq.setInt('/dev8721/awgs/0/time', sampling_index)

sampling_rate= 1/2**sampling_index

times_map = {'pulse_time': 50e-3,
             'trigger_time': 1e-3,
             'wait_time': 1e-3} # The time needs to be larger than the ...

for key in times_map:
    times_map[key] /= 2**sampling_index
print(times_map)

{'pulse_time': 6.103515625e-06, 'trigger_time': 1.220703125e-07, 'wait_time': 1.220703125e-07}


In [35]:
#Will use this for the second version
START = -0.1; STOP = 0.1; STEPS = 50

## define length sweep parameter
level_sweep_parameter = LinearSweepParameter(uid="JoJo", start=START, stop=STOP, count=STEPS)
#level_sweep_parameter_2 = LinearSweepParameter(uid="LaLa", start=START, stop=STOP, count=STEPS)
#we might even be able to reference the same sweep if they're the same values 

#### Define pulse shapes
@pulse_library.register_pulse_functional


In [36]:
level_pulse = pulse_library.const(uid="step_level",length=times_map['pulse_time'],amplitude=1)
compress_level_pulse = pulse_library.const(uid="compress_step_level",length=times_map['pulse_time'],amplitude=1,can_compress=True)

#we have to change some step size bc when i compress it, we have pulses of varying size

In [37]:
## Create Experiment
exp = Experiment(
    signals=[
        ExperimentSignal("gate3"), #or 1 and 2 
        ExperimentSignal("gate4"),
    ],
)

# define experiment
NUM_REP = 1
AmplitudeZigZag = [0.005, -0.005, 0.006, -0.006, 0.007, -0.007, 0.008, -0.008, 0.009, -0.009 ,0.01,-0.01]

with exp.acquire_loop_rt(
    uid=("sweep"), count=NUM_REP, averaging_mode=AveragingMode.SEQUENTIAL #changed uid to sweep 
):
    with exp.sweep(uid='sweep_rabi',parameter=level_sweep_parameter): #parameter=[level_sweep_parameter, level_sweep_parameter_2]): 
        with exp.section(
            uid="between_trigger", 
            length=times_map['trigger_time'],
            alignment=SectionAlignment.LEFT):
            exp.reserve(signal="gate3")
        with exp.section(
            uid=("level1"),
            length=times_map['pulse_time']*15,
            trigger={"gate3": {"state": 1}},
            alignment=SectionAlignment.LEFT,
        ):
            for i in range(len(AmplitudeZigZag)):
                exp.play(signal="gate3", pulse=level_pulse, amplitude = AmplitudeZigZag[i]) 
                exp.play(signal="gate4", pulse=level_pulse, amplitude = AmplitudeZigZag[i])
                exp.delay(signal="gate3", time=times_map['wait_time'])
                exp.delay(signal="gate4", time=times_map['wait_time'])



In [38]:
# shortcut to the logical signal group q0
lsg = device_setup.logical_signal_groups["q0"].logical_signals

# define signal map
map_signals = {
    "gate3" : lsg["fg8_line"],
    "gate4" : lsg["fg9_line"]
}

#Maybe we have to wait here???

In [39]:
source = ps.PS5000A_CHANNEL["PS5000A_CHANNEL_B"] #set the buffer
noOfCaptures = NUM_REP#*STEPS #simply NUM_REP in 2 Level pulse 
status,timeIntervalns = pm.capture_rapid_data_block(chandle, preTriggerSamples, postTriggerSamples, timebase, maxSamples,noOfCaptures)


# # create and connect to session
session = Session(device_setup=device_setup)
session.connect(do_emulation=DO_EMULATION)
# set experiment calibration and signal map
exp.set_signal_map(map_signals)


if not session.connection_state.emulated:
    instrument_serial = device_setup.instrument_by_uid("device_hdawg").address
    device = session.devices[instrument_serial]
    device.triggers.out[2].delay(23.9e-9)
print("Loaded exp")
session.run(exp)
print("Running exp finished")
# Check for data collection to finish using ps5000aIsReady
while ready.value == check.value:
    status["isReady"] = ps.ps5000aIsReady(chandle, ctypes.byref(ready))

buffersMax, buffersMin = pm.create_rapid_buffer(chandle, source, maxSamples, noOfCaptures)
pm.getValuesRapid(chandle,maxSamples,noOfCaptures)

2023.08.10 13:49:08.944 laboneq.controller.laboneq_logging INFO   Logging initialized from [Default inline config in laboneq.controller.laboneq_logging] logdir is c:\Users\B15measure2\HDAWG\laboneq_output\log
2023.08.10 13:49:08.947 laboneq.controller.controller  INFO   VERSION: laboneq 2.11.0
2023.08.10 13:49:08.952 laboneq.controller.devices.device_collection INFO   Connecting to data server at 127.0.0.1:8004


2023.08.10 13:49:09.047 laboneq.controller.communication INFO   Connected to Zurich Instruments LabOne Data Server version 23.02 at 127.0.0.1:8004
2023.08.10 13:49:09.529 laboneq.controller.devices.device_collection INFO   Configuring the device setup
2023.08.10 13:49:09.711 laboneq.controller.devices.device_collection INFO   The device setup is configured
Loaded exp
2023.08.10 13:49:09.944 laboneq.compiler.workflow.compiler INFO   Starting LabOne Q Compiler run...
2023.08.10 13:49:10.305 laboneq.compiler.scheduler.scheduler INFO   Schedule completed
2023.08.10 13:49:10.748 laboneq.compiler.workflow.compiler INFO   Total seqC lines generated: 131
2023.08.10 13:49:10.749 laboneq.compiler.workflow.compiler INFO   Total sample points generated: 351904
2023.08.10 13:49:10.751 laboneq.compiler.workflow.compiler INFO   Finished LabOne Q Compiler run.
2023.08.10 13:49:10.977 laboneq.controller.controller  INFO   Starting near-time execution...
2023.08.10 13:49:13.090 laboneq.controller.contro

{'GetValuesBulk': 0}

In [40]:
# convert ADC counts data to mV
cmaxSamples = ctypes.c_int32(maxSamples)

# Create time data
time_array = np.linspace(0, (cmaxSamples.value - 1) * timeIntervalns.value, cmaxSamples.value)
voltage_in_mV=[]
with open('Data/ZigZag_%s.npy'%(int(time.time())), 'wb') as f:
    for i in range(noOfCaptures):
        # plot data from channel A and B
        if i == 0:
            voltage_in_mV = adc2mV(buffersMax[i], chBRange, maxADC)
        else:
            voltage_in_mV=np.vstack([voltage_in_mV, adc2mV(buffersMax[i], chBRange, maxADC)])
    np.save(f,voltage_in_mV)

# display status returns
# print(status)

# Stop the scope
status["stop"] = ps.ps5000aStop(chandle)
assert_pico_ok(status["stop"])


# # Close unit Disconnect the scope
status["close"]=ps.ps5000aCloseUnit(chandle)
assert_pico_ok(status["close"])

In [41]:
#sample points are still of def lengths, which is weird 

https://docs.zhinst.com/labone_q_user_manual/concepts/averaging_sweeping/